# 训练 CLIP 文本到图像嵌入的映射网络

1. 环境设置
首先，导入必要的库，并配置一些基本参数。

In [2]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import random
import numpy as np

# 设置随机种子以确保可重复性
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# 配置参数
TEXT_EMBEDDINGS_PATH = './clip_embeddings/text_embeddings.pt'   # 替换为实际路径
IMAGE_EMBEDDINGS_PATH = './clip_embeddings/image_embeddings.pt' # 替换为实际路径
MODEL_SAVE_PATH = 'text_to_image_embedder.pth'
BATCH_SIZE = 128
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4

# 设备设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")


使用设备: cuda


2. 定义数据集类
创建一个自定义的 Dataset 类，用于加载预先计算的文本和图像嵌入。

In [3]:
# 定义自定义 Dataset 类
class CLIPEmbedMappingDatasetPrecomputed(Dataset):
    def __init__(self, text_embeddings_path, image_embeddings_path):
        """
        初始化数据集，加载预计算的嵌入
        :param text_embeddings_path: 文本嵌入文件路径
        :param image_embeddings_path: 图像嵌入文件路径
        """
        # 加载嵌入
        try:
            self.text_embeddings = torch.load(text_embeddings_path)
            self.image_embeddings = torch.load(image_embeddings_path)
        except Exception as e:
            print(f"Error loading embeddings: {e}")
            raise

        # 确认嵌入数量匹配
        assert len(self.text_embeddings) == len(self.image_embeddings), "文本和图像嵌入的数量不匹配"

        # 确保嵌入的 dtype 为 float32
        if self.text_embeddings.dtype != torch.float32:
            self.text_embeddings = self.text_embeddings.float()
            print("已将文本嵌入转换为 float32")
        if self.image_embeddings.dtype != torch.float32:
            self.image_embeddings = self.image_embeddings.float()
            print("已将图像嵌入转换为 float32")

    def __len__(self):
        return len(self.text_embeddings)

    def __getitem__(self, idx):
        return self.text_embeddings[idx], self.image_embeddings[idx]


3. 定义映射网络模型
定义一个简单的全连接神经网络，将文本嵌入映射到图像嵌入空间。

In [4]:
# 定义映射网络模型
class TextToImageEmbedder(nn.Module):
    def __init__(self, clip_dim=512, embed_dim=512):
        super(TextToImageEmbedder, self).__init__()
        self.mapping = nn.Sequential(
            nn.Linear(clip_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim)
        )
    
    def forward(self, text_embeddings):
        image_embeddings = self.mapping(text_embeddings)
        return image_embeddings


4. 创建数据加载器
使用自定义的 Dataset 类创建 DataLoader，以便在训练过程中批量加载数据。

In [5]:
# 创建映射网络的数据集和数据加载器（使用预计算嵌入）
try:
    mapping_dataset = CLIPEmbedMappingDatasetPrecomputed(
        text_embeddings_path=TEXT_EMBEDDINGS_PATH,
        image_embeddings_path=IMAGE_EMBEDDINGS_PATH
    )
except Exception as e:
    print(f"Error initializing mapping dataset: {e}")
    raise

# 确认数据集大小
print(f"数据集大小: {len(mapping_dataset)}")

# 创建 DataLoader
mapping_loader = DataLoader(
    mapping_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,  # 根据需要调整
    pin_memory=True
)


/tmp/ipykernel_6208/2941295110.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.text_embeddings = torch.load(text_embeddings_path)
/tmp/ipykernel_6208/2941295110.py

已将文本嵌入转换为 float32
已将图像嵌入转换为 float32
数据集大小: 162770


5. 初始化模型、损失函数和优化器

In [6]:
# 创建映射网络模型
embedder = TextToImageEmbedder(clip_dim=512, embed_dim=512).to(device)

# 定义损失函数和优化器
criterion_mapping = nn.MSELoss()
optimizer_mapping = optim.Adam(embedder.parameters(), lr=LEARNING_RATE)


6. 训练映射网络
定义训练函数，并开始训练过程。

In [7]:
# 定义训练函数
def train_mapping_network(model, loader, optimizer, criterion, num_epochs=20):
    model.train()
    for epoch in range(1, num_epochs + 1):
        epoch_loss = 0
        for text_emb, img_emb in tqdm(loader, desc=f"Mapping Epoch {epoch}/{num_epochs}"):
            # 将嵌入移动到GPU
            text_emb = text_emb.to(device)
            img_emb = img_emb.to(device)

            # 前向传播
            pred_img_emb = model(text_emb)
            loss = criterion(pred_img_emb, img_emb)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(loader.dataset)
        print(f"Mapping Epoch {epoch}, 平均损失: {avg_loss:.6f}")
    return model

# 开始训练
trained_embedder = train_mapping_network(embedder, mapping_loader, optimizer_mapping, criterion_mapping, num_epochs=20)


Mapping Epoch 1/20: 100%|██████████| 1272/1272 [00:03<00:00, 413.40it/s]


Mapping Epoch 1, 平均损失: 0.000563


Mapping Epoch 2/20: 100%|██████████| 1272/1272 [00:02<00:00, 463.65it/s]


Mapping Epoch 2, 平均损失: 0.000514


Mapping Epoch 3/20: 100%|██████████| 1272/1272 [00:02<00:00, 471.60it/s]


...

Mapping Epoch 18, 平均损失: 0.000488


Mapping Epoch 19/20: 100%|██████████| 1272/1272 [00:02<00:00, 561.67it/s]


Mapping Epoch 19, 平均损失: 0.000487


Mapping Epoch 20/20: 100%|██████████| 1272/1272 [00:02<00:00, 499.27it/s]

Mapping Epoch 20, 平均损失: 0.000487


In [8]:
# 保存映射网络
torch.save(trained_embedder.state_dict(), MODEL_SAVE_PATH)
print(f"映射网络已保存为: {MODEL_SAVE_PATH}")


映射网络已保存为: text_to_image_embedder.pth


## 利用后来生成的嵌入训练了一个新的网络效果比较好

In [6]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import random
import numpy as np

# 设置随机种子以确保可重复性
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# 配置参数
TEXT_EMBEDDINGS_PATH = '/root/autodl-tmp/clip_embeddings/text_embeddings_partition_0.pt'   # 替换为实际路径
IMAGE_EMBEDDINGS_PATH = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt' # 替换为实际路径
MODEL_SAVE_PATH = 'text_to_image_embedder.pth'
BATCH_SIZE = 128
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4

# 设备设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 定义自定义 Dataset 类
class CLIPEmbedMappingDatasetPrecomputed(Dataset):
    def __init__(self, text_embeddings_path, image_embeddings_path):
        """
        初始化数据集，加载预计算的嵌入
        :param text_embeddings_path: 文本嵌入文件路径
        :param image_embeddings_path: 图像嵌入文件路径
        """
        # 加载嵌入
        try:
            self.text_embeddings = torch.load(text_embeddings_path)
            self.image_embeddings = torch.load(image_embeddings_path)
        except Exception as e:
            print(f"Error loading embeddings: {e}")
            raise

        # 确认嵌入数量匹配
        assert len(self.text_embeddings) == len(self.image_embeddings), "文本和图像嵌入的数量不匹配"

        # 确保嵌入的 dtype 为 float32
        if self.text_embeddings.dtype != torch.float32:
            self.text_embeddings = self.text_embeddings.float()
            print("已将文本嵌入转换为 float32")
        if self.image_embeddings.dtype != torch.float32:
            self.image_embeddings = self.image_embeddings.float()
            print("已将图像嵌入转换为 float32")

    def __len__(self):
        return len(self.text_embeddings)

    def __getitem__(self, idx):
        return self.text_embeddings[idx], self.image_embeddings[idx]

# 定义优化后的映射网络模型
class TextToImageEmbedder(nn.Module):
    def __init__(self, clip_dim=512, embed_dim=512):
        super(TextToImageEmbedder, self).__init__()
        self.mapping = nn.Sequential(
            nn.Linear(clip_dim, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, embed_dim),
            nn.BatchNorm1d(embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim)
        )
    
    def forward(self, text_embeddings):
        image_embeddings = self.mapping(text_embeddings)
        return image_embeddings

# 定义新的余弦相似度损失函数
class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()
        self.cos = nn.CosineSimilarity(dim=1)
    
    def forward(self, pred, target):
        # 余弦相似度范围在 [-1, 1]，我们希望最大化相似度，因此损失为 1 - cos_sim
        return 1 - self.cos(pred, target).mean()

# 创建映射网络的数据集和数据加载器（使用预计算嵌入）
try:
    mapping_dataset = CLIPEmbedMappingDatasetPrecomputed(
        text_embeddings_path=TEXT_EMBEDDINGS_PATH,
        image_embeddings_path=IMAGE_EMBEDDINGS_PATH
    )
except Exception as e:
    print(f"Error initializing mapping dataset: {e}")
    raise

# 确认数据集大小
print(f"数据集大小: {len(mapping_dataset)}")

# 创建 DataLoader
mapping_loader = DataLoader(
    mapping_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,  # 根据需要调整
    pin_memory=True
)

# 创建映射网络模型
embedder = TextToImageEmbedder(clip_dim=512, embed_dim=512).to(device)

# 定义损失函数和优化器
criterion_mapping = CosineSimilarityLoss()
optimizer_mapping = optim.Adam(embedder.parameters(), lr=LEARNING_RATE)

# 定义训练函数
def train_mapping_network(model, loader, optimizer, criterion, num_epochs=20):
    model.train()
    for epoch in range(1, num_epochs + 1):
        epoch_loss = 0
        for text_emb, img_emb in tqdm(loader, desc=f"Mapping Epoch {epoch}/{num_epochs}"):
            # 将嵌入移动到GPU
            text_emb = text_emb.to(device)
            img_emb = img_emb.to(device)

            # 前向传播
            pred_img_emb = model(text_emb)
            loss = criterion(pred_img_emb, img_emb)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(loader.dataset)
        print(f"Mapping Epoch {epoch}, 平均损失: {avg_loss:.6f}")
    return model

# 开始训练
trained_embedder = train_mapping_network(embedder, mapping_loader, optimizer_mapping, criterion_mapping, num_epochs=NUM_EPOCHS)

# 保存映射网络
torch.save(trained_embedder.state_dict(), MODEL_SAVE_PATH)
print(f"映射网络已保存为: {MODEL_SAVE_PATH}")


使用设备: cuda


/tmp/ipykernel_14605/393585030.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.text_embeddings = torch.load(text_embeddings_path)
/tmp/ipykernel_14605/393585030.py

已将文本嵌入转换为 float32
已将图像嵌入转换为 float32
数据集大小: 162770


Mapping Epoch 1/20: 100%|██████████| 1272/1272 [00:04<00:00, 301.80it/s]


Mapping Epoch 1, 平均损失: 0.001742


...

Mapping Epoch 18, 平均损失: 0.001338


Mapping Epoch 19/20: 100%|██████████| 1272/1272 [00:03<00:00, 323.90it/s]


Mapping Epoch 19, 平均损失: 0.001336


Mapping Epoch 20/20: 100%|██████████| 1272/1272 [00:03<00:00, 322.61it/s]

Mapping Epoch 20, 平均损失: 0.001335
映射网络已保存为: text_to_image_embedder.pth


## 下面这个映射函数太烂了 训练出来的是个屎

In [5]:
# 导入必要的库
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random
import numpy as np

# 设置随机种子以确保可重复性
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# 配置参数
TEXT_EMBEDDINGS_PATH = '/root/autodl-tmp/clip_embeddings/text_embeddings_partition_0.pt'   # 替换为实际路径
IMAGE_EMBEDDINGS_PATH = '/root/autodl-tmp/clip_embeddings/image_embeddings_partition_0.pt' # 替换为实际路径
MODEL_SAVE_PATH = 'text_to_image_embedder.pth'
BATCH_SIZE = 128
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4

# 设备设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 定义自定义 Dataset 类
class CLIPEmbedMappingDatasetPrecomputed(Dataset):
    def __init__(self, text_embeddings_path, image_embeddings_path):
        """
        初始化数据集，加载预计算的嵌入
        :param text_embeddings_path: 文本嵌入文件路径
        :param image_embeddings_path: 图像嵌入文件路径
        """
        # 加载嵌入
        try:
            self.text_embeddings = torch.load(text_embeddings_path)
            self.image_embeddings = torch.load(image_embeddings_path)
        except Exception as e:
            print(f"Error loading embeddings: {e}")
            raise

        # 确认嵌入数量匹配
        assert len(self.text_embeddings) == len(self.image_embeddings), "文本和图像嵌入的数量不匹配"

        # 确保嵌入的 dtype 为 float32
        if self.text_embeddings.dtype != torch.float32:
            self.text_embeddings = self.text_embeddings.float()
            print("已将文本嵌入转换为 float32")
        if self.image_embeddings.dtype != torch.float32:
            self.image_embeddings = self.image_embeddings.float()
            print("已将图像嵌入转换为 float32")

    def __len__(self):
        return len(self.text_embeddings)

    def __getitem__(self, idx):
        return self.text_embeddings[idx], self.image_embeddings[idx]

# 使用GELU或SiLU(Swish)激活函数
# PyTorch中SiLU基本等同于Swish: y = x * sigmoid(x)
# 我们这里使用nn.SiLU()来代替GELU，如果需要GELU可以将nn.SiLU()替换为nn.GELU()
activation_fn = nn.SiLU()

# 定义改进的映射网络模型
class TextToImageEmbedder(nn.Module):
    def __init__(self, clip_dim=512, embed_dim=512):
        super(TextToImageEmbedder, self).__init__()
        # 增加层数和宽度，并使用SiLU激活函数来替换ReLU
        self.mapping = nn.Sequential(
            nn.Linear(clip_dim, 4096),
            nn.BatchNorm1d(4096),
            activation_fn,
            nn.Dropout(0.2),
            nn.Linear(4096, 2048),
            nn.BatchNorm1d(2048),
            activation_fn,
            nn.Dropout(0.2),
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            activation_fn,
            nn.Dropout(0.2),
            nn.Linear(1024, embed_dim),
            nn.BatchNorm1d(embed_dim),
            activation_fn,
            nn.Linear(embed_dim, embed_dim) # 输出层
        )

    def forward(self, text_embeddings):
        image_embeddings = self.mapping(text_embeddings)
        # 可选：对映射后的图像嵌入进行归一化，使其分布更稳定
        image_embeddings = image_embeddings / (image_embeddings.norm(dim=1, keepdim=True) + 1e-8)
        return image_embeddings

# 定义对比损失 - 使用三元组损失作为简单的对比学习示例
# anchor: 映射网络预测的图像嵌入 pred_img_emb
# positive: 对应的真实图像嵌入 img_emb[i]
# negative: 来自同一batch中其他样本的img_emb
triplet_loss_fn = nn.TripletMarginLoss(margin=0.2)

def contrastive_loss(pred_img_emb, img_emb):
    """
    对比损失函数(简易版)：使用三元组损失进行对比学习
    对于每个样本(i):
      anchor = pred_img_emb[i]
      positive = img_emb[i]
      negative = img_emb[j], j != i

    需要保证batch_size > 1
    """
    batch_size = pred_img_emb.size(0)
    # 随机选择negative样本
    # 为了简单起见，从同一batch中随机选取与i不同的样本作为negative
    indices = torch.arange(batch_size, device=pred_img_emb.device)
    # 打乱indices
    shuffled = indices[torch.randperm(batch_size)]
    # 确保没有出现与i相同的索引
    # 如果出现相同索引则再次随机
    # 简化处理：若有相同索引则重排一次
    for i in range(batch_size):
        if shuffled[i] == i:
            # 简单处理：将该位置与下一个位置交换
            if i < batch_size - 1:
                shuffled[i], shuffled[i+1] = shuffled[i+1], shuffled[i]
            else:
                shuffled[i], shuffled[i-1] = shuffled[i-1], shuffled[i]

    negative_img_emb = img_emb[shuffled]
    loss = triplet_loss_fn(pred_img_emb, img_emb, negative_img_emb)
    return loss

# 创建映射网络的数据集和数据加载器（使用预计算嵌入）
try:
    mapping_dataset = CLIPEmbedMappingDatasetPrecomputed(
        text_embeddings_path=TEXT_EMBEDDINGS_PATH,
        image_embeddings_path=IMAGE_EMBEDDINGS_PATH
    )
except Exception as e:
    print(f"Error initializing mapping dataset: {e}")
    raise

# 确认数据集大小
print(f"数据集大小: {len(mapping_dataset)}")

# 创建 DataLoader
mapping_loader = DataLoader(
    mapping_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,  # 根据需要调整
    pin_memory=True
)

# 创建映射网络模型
embedder = TextToImageEmbedder(clip_dim=512, embed_dim=512).to(device)

# 定义优化器
optimizer_mapping = optim.Adam(embedder.parameters(), lr=LEARNING_RATE)

# 定义训练函数
def train_mapping_network(model, loader, optimizer, num_epochs=NUM_EPOCHS):
    model.train()
    for epoch in range(1, num_epochs + 1):
        epoch_loss = 0
        for text_emb, img_emb in tqdm(loader, desc=f"Mapping Epoch {epoch}/{num_epochs}"):
            # 将嵌入移动到GPU
            text_emb = text_emb.to(device)
            img_emb = img_emb.to(device)

            # 前向传播
            pred_img_emb = model(text_emb)

            # 使用对比损失函数(三元组损失)
            loss = contrastive_loss(pred_img_emb, img_emb)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(loader.dataset)
        print(f"Mapping Epoch {epoch}, 平均训练损失: {avg_loss:.6f}")
    return model

# 开始训练
trained_embedder = train_mapping_network(embedder, mapping_loader, optimizer_mapping, num_epochs=NUM_EPOCHS)

# 保存映射网络
torch.save(trained_embedder.state_dict(), MODEL_SAVE_PATH)
print(f"映射网络已保存为: {MODEL_SAVE_PATH}")


使用设备: cuda


/tmp/ipykernel_14605/3116050070.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.text_embeddings = torch.load(text_embeddings_path)
/tmp/ipykernel_14605/3116050070.

已将文本嵌入转换为 float32
已将图像嵌入转换为 float32
数据集大小: 162770


Mapping Epoch 1/20: 100%|██████████| 1272/1272 [00:09<00:00, 140.66it/s]


Mapping Epoch 1, 平均训练损失: 0.002054


...

Mapping Epoch 18, 平均训练损失: 0.001895


Mapping Epoch 19/20: 100%|██████████| 1272/1272 [00:09<00:00, 139.76it/s]


Mapping Epoch 19, 平均训练损失: 0.001898


Mapping Epoch 20/20: 100%|██████████| 1272/1272 [00:08<00:00, 148.42it/s]

Mapping Epoch 20, 平均训练损失: 0.001895
映射网络已保存为: text_to_image_embedder.pth
